# Sid's xA^TA
To run this script, write a .mat file from matlab which contain all the node coordinates of the two faces 

In [28]:



import numpy as np
import scipy.io as sio
from numpy.linalg import norm
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import BFGS

# # NID_An4 = sio.loadmat('180508_coordsFace1.mat')['thisFaceNodes_An4']
# NC_An4 = sio.loadmat('face100.mat')['face_node_coord_an4']
# # NID_An5 = sio.loadmat('180508_coordsFace1.mat')['thisFaceNodes_An5']
# NC_An5 = sio.loadmat('face100.mat')['face_node_coord_an5']

NC_An4 = np.array([[0,0,0], [0,1,0], [1,0,0], [1,1,0], [2,2,0]]);
NC_An5 = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1],[2,2,1]]);

m = NC_An4.shape[0];
n = NC_An5.shape[0];

x = np.repeat(NC_An4.reshape(m,1,3), n, axis=1);
y = np.repeat(NC_An5.reshape(n,1,3).swapaxes(0,1), m, axis=0);
if np.sum(x[:,0,:] == NC_An4) != m*3:
    print 'x shape wrong!'
elif np.sum(y[0,:,:] == NC_An5) != n*3:
    print 'y shape wrong!'
# coeff = norm(x-y, axis=2)
coeff = norm(x-y, axis=2)**2
coeff = np.reshape(coeff, m*n, )

bounds = Bounds(np.zeros((m*n, )), np.ones((m*n, ))*np.inf)

constraints = np.zeros((m*n, m+n))
for i in range(m):
    constraints[i*n : i*n+n, i] = 1
for i in range(n):
    constraints[np.arange(i, m*n, n), m+i] = 1
# print constrains[:, m+n-1].reshape(m,n)
lims = np.hstack((np.ones((m,)), np.ones((n,))*m/n))

linear_constraint = LinearConstraint(constraints.T, lims, lims)

weights_init = (np.ones((m,n))*1/n).reshape(m*n,)

obj = lambda weights: weights.T.dot(coeff)

# res = minimize(obj, weights_init, constraints=[linear_constraint], bounds=bounds, 
#                method='trust-constr', jac="2-point", hess=BFGS(), options={'gtol': 1e-4, 'disp': True})

def makeJac(x):
    return coeff
def makeHess(x):
    return np.zeros((coeff.shape[0], coeff.shape[0]))

res = minimize(obj, weights_init, constraints=[linear_constraint], bounds=bounds, 
               method='trust-constr', jac=makeJac, hess=makeHess, options={'gtol': 1e-3, 'disp': True})



`gtol` termination condition is satisfied.
Number of iterations: 10, function evaluations: 9, CG iterations: 31, optimality: 3.03e-04, constraint violation: 2.44e-15, execution time: 0.058 s.


In [30]:
constraints

array([[1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 

In [12]:
np.sum(res.x.reshape(m,n), axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])